In [1]:
import os

try:
    from dotenv import load_dotenv
except ImportError:
    from pip._internal import main as pip
    pip(['install', 'python-dotenv'])
    from dotenv import load_dotenv

try:
    import psycopg2
except ImportError:
    from pip._internal import main as pip
    pip(['install', 'psycopg2'])
    import psycopg2

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [2]:
load_dotenv(dotenv_path='//.env')
db = os.getenv('DB')

In [137]:
acled = pd.read_sql_query('SELECT sub_event_type, COUNT(*) FROM acled GROUP BY sub_event_type ORDER BY COUNT(*) desc',db)
sipri = pd.read_sql_query('SELECT weapon_description, SUM(number_ordered) as total FROM sipri GROUP BY weapon_description HAVING SUM(number_ordered) IS NOT NULL ORDER BY total desc',db)
#acled = pd.read_sql_query('SELECT DISTINCT actor1 FROM acled', db)
#sipri = pd.read_sql_query('SELECT DISTINCT buyer FROM sipri', db)

In [138]:
acled_counts = px.bar(acled,x='sub_event_type',y='count')
acled_counts.show()

In [139]:
#sipri_filter = sipri.query('total > 440')
sipri_counts = px.bar(sipri,x='weapon_description',y='total')
sipri_counts.show()

In [224]:
#acled_countries = pd.read_sql_query('SELECT country,sub_event_type, COUNT(*) FROM acled WHERE year > 2015 GROUP BY country,sub_event_type ORDER BY COUNT(*) DESC',db)
acled_countries = pd.read_sql_query('SELECT country, COUNT(*) FROM acled WHERE year > 2015 GROUP BY country ORDER BY COUNT(*) DESC',db)
#sipri_countries = pd.read_sql_query('SELECT buyer, SUM(number_ordered) as total FROM sipri WHERE CAST(order_year as integer) > 2015 GROUP BY buyer HAVING SUM(number_ordered) IS NOT NULL ORDER BY total desc',db)
sipri_countries = pd.read_sql_query('SELECT buyer, weapon_description, SUM(number_delivered) as total FROM sipri WHERE CAST(order_year as integer) > 2015 GROUP BY buyer, weapon_description HAVING SUM(number_delivered) IS NOT NULL ORDER BY total desc',db)

In [201]:
acled_conflict_countries = px.bar(acled_countries,x='country',y='count',
title='Total Amount of Conflict Events Amongst<br>Nations Across All Categories 2015-2020',labels={'country':'Nation','count':'Counts'})
acled_conflict_countries.show()

In [199]:
acled_conflict_countries = px.bar(acled_countries.query('country=="India"|country=="Syria"|country=="Ukraine"|country=="Yemen"|country=="Afghanistan"'),x='country',y='count',color='sub_event_type',
title='Type of Conflict For Top 5 Nations With Most Conflict 2015-2020',
labels={'country':'Nation','count':'Counts', 'sub_event_type':'Conflict Type'},
category_orders={'country':['India','Syria','Yemen','Afghanistan','Ukraine']})
acled_conflict_countries.show()

In [227]:
sipri_most_purchases = px.bar(sipri_countries,x='buyer',y='total',
title="Total Amount of Weapon Purchases by Nation 2015-2020",
labels={'buyer':'Nation or Non-State Actor','total':'Amount of Purchases'})
sipri_most_purchases.show()

In [226]:
sipri_most_purchases = px.bar(sipri_countries.query('buyer=="India"|buyer=="Saudi Arabia"|buyer=="United Arab Emirates"|buyer=="Afghanistan"|buyer=="Qatar"'),x='buyer',y='total', color='weapon_description',
title="Total Amount of Weapon Purchases by Top 5 Nations 2015-2020",
labels={'buyer':'Nation or Non-State Actor','total':'Amount of Purchases'},
category_orders={'buyer':['Saudi Arabia','United Arab Emirates','India','Afghanistan','Qatar']}
)
sipri_most_purchases.show()

In [304]:
#acled_countries = pd.read_sql_query('SELECT country,sub_event_type, COUNT(*) FROM acled GROUP BY country,sub_event_type ORDER BY COUNT(*) DESC',db)
#acled_countries = pd.read_sql_query("""SELECT country,sub_event_type, COUNT(*) FROM acled WHERE sub_event_type <>'Peaceful protest' AND sub_event_type <> 'Change to group/activity' GROUP BY country,sub_event_type ORDER BY COUNT(*) DESC""",db)
acled_countries = pd.read_sql_query("""SELECT country, COALESCE(sub_event_type,'Total') AS event, total FROM (SELECT 
	country,sub_event_type,COUNT(sub_event_type) as total
FROM
	acled
GROUP BY
	ROLLUP(country,sub_event_type)
ORDER BY
	country,total) AS rolledup""",db)

In [305]:
#acled_countries.describe(include='all')
#df_test = pd.DataFrame(index=pd.MultiIndex.from_frame(acled_countries))
newdf = acled_countries.set_index(['country','event']).sort_index()

In [309]:
newdf.query('country=="India"').sort_values(by=['country','total'])

total
country event                                    
India   Headquarters or base established        1
        Non-violent transfer of territory       1
        Air/drone strike                        2
        Government regains territory            2
        Suicide bomb                            3
        Agreement                              33
        Other                                  38
        Change to group/activity               47
        Shelling/artillery/missile attack      62
        Sexual violence                        89
        Abduction/forced disappearance        146
        Arrests                               159
        Excessive force against protesters    194
        Looting/property destruction          302
        Grenade                               346
        Disrupted weapons use                 431
        Remote explosive/landmine/IED         692
        Attack                               3601
        Protest with intervention            4474
        Armed clash                          4789
        Violent demonstration                4933
        Mob violence                         8630
        Peaceful protest                    60693
        Total                               89668

In [129]:
acled_drones = pd.read_sql_query("""SELECT country,
COALESCE("2000",0)"2000",
COALESCE("2001",0) "2001",
COALESCE("2002",0)"2002",
COALESCE("2003",0)"2003",
COALESCE("2004",0)"2004",
COALESCE("2005",0)"2005",
COALESCE("2006",0)"2006",
COALESCE("2007",0)"2007",
COALESCE("2008",0)"2008",
COALESCE("2009",0)"2009",
COALESCE("2010",0)"2010",
COALESCE("2011",0)"2011",
COALESCE("2012",0)"2012",
COALESCE("2013",0)"2013",
COALESCE("2014",0)"2014",
COALESCE("2015",0)"2015",
COALESCE("2016",0)"2016",
COALESCE("2017",0)"2017",
COALESCE("2018",0)"2018",
COALESCE("2019",0)"2019",
COALESCE("2020",0)"2020"
FROM crosstab($$
	SELECT country,year,COUNT(*) AS ct
	FROM acled
	WHERE
	notes ILIKE '%drone%'
			  AND
			  sub_event_type IN ('Shelling/artillery/missile attack',
				   'Armed clash',
				   'Disrupted weapons use',
				   'Air/drone strike'
				  )
	GROUP BY 1,2
	ORDER BY 1$$,
	$$ SELECT y FROM generate_series(2000,2020) y$$) 
AS ct(country text,
	  "2000" bigint,
	 "2001" bigint,
	 "2002" bigint,
	 "2003" bigint,
	 "2004" bigint,
	 "2005" bigint,
	 "2006" bigint,
	 "2007" bigint,
	 "2008" bigint,
	 "2009" bigint,
	 "2010" bigint,
	 "2011" bigint,
	 "2012" bigint,
	 "2013" bigint,
	 "2014" bigint,
	 "2015" bigint,
	 "2016" bigint,
	 "2017" bigint,
	 "2018" bigint,
	 "2019" bigint,
	 "2020" bigint)""",db)

In [144]:
df5 = acled_drones.set_index('country').T

In [139]:
acled_drones.iloc[0][1:22]

2000      0
2001      0
2002      0
2003      0
2004      0
2005      0
2006      0
2007      0
2008      0
2009      0
2010      0
2011      0
2012      0
2013      0
2014      0
2015      0
2016      0
2017    190
2018    125
2019    128
2020     42
Name: 0, dtype: object

In [154]:
acled_drones_graph = px.scatter(acled_drones,x=acled_drones['country'],y=acled_drones.columns[1:],
title="Total Amount of Weapon Purchases by Top 5 Nations 2015-2020",
labels={'buyer':'Nation or Non-State Actor','total':'Amount of Purchases'}
)
acled_drones_graph.show()

In [93]:
#acled_countries = list(acled['actor1'])
#sipri_countries = list(sipri['buyer'])

In [94]:
#res = [x for x in acled_countries + sipri_countries if x not in sipri_countries]
#print(sorted(res, key=str.lower))

In [95]:
#res = [x for x in acled_countries + sipri_countries if x not in acled_countries]
#print(sorted(res, key=str.lower))